In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
from datetime import datetime, timedelta

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()
import pprint

# Add a query tag to the session.
session.query_tag = {"origin":"sf_sit-is", "name":"cortex_support_case_analysis", "version":{"major":1, "minor":0}}

import snowflake.snowpark.functions as F
from IPython.display import Markdown, display
# model="mixtral-8x7b"

pp = pprint.PrettyPrinter(indent=4)
model="mistral-large"



 Setup and Load support tickets in CSV format to stage
 Then we will create a external S3 stage for the supportcases.csv file stored in a public S3 bucket.

Use the Snowpark DataFrame Reader to read in data from the externally staged supportcases CSV file
In setup.sql, we staged the supportcases.csv file from an external s3 bucket. Now, we can read it in.For more information on loading data, see documentation on snowflake.snowpark.DataFrameReader.

In [ ]:
-- Create csv format
CREATE FILE FORMAT IF NOT EXISTS CSVFORMAT 
    SKIP_HEADER = 1 
    TYPE = 'CSV';

-- Create external stage with the csv format to stage the diamonds dataset
CREATE STAGE IF NOT EXISTS SUPPORT_CASES_STAGE
    FILE_FORMAT =  CSVFORMAT 
    URL = 's3://sfquickstarts/sfguide_analyzing_support_cases_using_snowflake_cortex/SUPPORT_CASES.csv';
    
-- Inspect content of stage
LS @SUPPORT_CASES_STAGE;


In [ ]:
 # Create a Snowpark DataFrame that is configured to load data from the CSV file
# We can now infer schema from CSV files.
support_tickets = session.read.options({"field_delimiter": ",",
                                    "field_optionally_enclosed_by": '"',
                                    "infer_schema": True,
                                    "parse_header": True}).csv("@SUPPORT_CASES_STAGE")

support_tickets

In [ ]:
support_tickets.write.save_as_table("SUPPORT_CASES", mode="overwrite")

In [ ]:
latest_date = max(support_tickets.select('DATE_CLOSED').collect())[0]
todays_date = datetime.now()
print(latest_date)
print(todays_date)

In [ ]:
diff_days = (todays_date - latest_date).days - 1
print(diff_days)

In [ ]:
session.sql(f"""
UPDATE SUPPORT_CASES
SET 
    DATE_CREATED = DATEADD(DAY, {diff_days}, DATE_CREATED),
    DATE_CLOSED = DATEADD(DAY, {diff_days}, DATE_CLOSED);
""").collect()

In [ ]:
support_tickets = session.table("SUPPORT_CASES")

In [ ]:
import streamlit as st

cases_df = support_tickets.select(
    F.concat(
        F.lit("##### \nCASE TITLE: "), 
        F.col("CASE_TITLE"), 
        F.lit("\n\nCASE DESCRIPTION: "), 
        F.col("CASE_DESCRIPTION"), 
        F.lit('\n\nCASE STATUS: '), 
        F.col("STATUS"), 
        F.lit('\n\nLAST COMMENT: '),
        F.col('LAST_UPDATE')
    ).alias('CASE_STRING'))
# cases_array_pd = cases_df.to_snowpark_pandas()
# cases_array_pd
display(Markdown(cases_df.collect()[1]['CASE_STRING'])) 

In [ ]:
num_tokens = cases_df.select(F.array_size(F.split(F.col('CASE_STRING'), F.lit(' '))).alias('num_tokens'))
num_tokens.agg(F.sum(F.col('num_tokens')).alias('total_words')).show()

In [ ]:
pandas_df = cases_df.to_pandas()

# Convert the Pandas DataFrame to a single appended string
appended_string = pandas_df.apply(lambda x: ' '.join(x.astype(str)), axis=1).str.cat(sep=' ')
pp.pprint(appended_string[:100])

In [ ]:
stmt1 = f'''create stage if not exists ENCODER_STAGE'''
session.sql(stmt1).collect()

## Add the file to the stage

In [ ]:
-- Inspect content of stage
LS @ENCODER_STAGE;


In [ ]:
from langchain_text_splitters import CharacterTextSplitter
import tiktoken
import pickle
import os

get_result = session.file.get("@ENCODER_STAGE/", "/tmp")
# Load the tiktoken ssh cache 
tiktoken_cache_dir = f'/tmp/'
os.environ["TIKTOKEN_CACHE_DIR"] = tiktoken_cache_dir


tiktokenfile = '9b5ad71b2ce5302211f9c61530b329a4922fc6a4'
if not os.path.exists(os.path.join(tiktoken_cache_dir, tiktokenfile)):
    raise FileNotFoundError(f"Cache file {tiktokenfile} not found in {tiktoken_cache_dir}")
encoding = tiktoken.get_encoding('cl100k_base')

def num_tokens_from_string(string: str) -> int:
    encoding = tiktoken.get_encoding('cl100k_base')
    num_tokens = len(encoding.encode(string))
    return num_tokens


text_splitter = CharacterTextSplitter().from_tiktoken_encoder(
    'cl100k_base',
    separator="#####",
    chunk_size=20000,
    chunk_overlap=4000,
    is_separator_regex=False,)


texts = text_splitter.create_documents([appended_string])

print("First 100 characters of first chunk:", str(texts[0])[:100])
print("Tokens in text:", num_tokens_from_string(str(texts[0])))


In [ ]:
from typing import Any, Dict, List, Optional
import time
import json

from snowflake.cortex import Complete
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM


class CortexLLM(LLM):
    max_retries = 1
    retry_delay = 10
    model = "reka-core"
    
    def _call(
            self,
            prompt: str,
            stop: Optional[List[str]] = None,
            run_manager: Optional[CallbackManagerForLLMRun] = None,
            **kwargs: Any,
        ) -> str:
            response = Complete(model, prompt)
            return response

    @property
    def _identifying_params(self) -> Dict[str, Any]:
        """Return a dictionary of identifying parameters."""
        return {
            "model_name": "CustomCortexModel",
        }

    @property
    def _llm_type(self) -> str:
        return "cortex"

In [ ]:
from langchain_core.prompts import PromptTemplate

map_template = PromptTemplate.from_template("""
                                            Given the following support cases for an order, return a summary of each case.
                                            Include details on the category of the issue, the errors or symptoms the customer noticed,
                                            and any basic details about what the customer was looking to accomplish.
                                            If multiple cases exist in the same category, you can group them together.
                                            The summary will be used to understand overall case trends and causes that the team can 
                                            use to prioritize fixes and improvements.
                                                
                                            ### Cases ###
                                            
                                            {cases}
                                                """)

reduce_template = PromptTemplate.from_template("""
                                                Given the following set of summaries for support case reports opened for an order, 
                                                distill it into a final, consolidated and detailed summary of trends and top pain points or blockers customers have been hitting.
                                                Prioritize issue categories that show up in multiple summaries as they are likely to be the most impactful.
                                                Include a description of the issue, the symptoms the customer noticed, what they were trying to do, and what led them to open the case.
                                               
                                                ### Case Chunk Summaries ###
                                               
                                                {summaries} 
                                               """)

In [ ]:
from langchain.chains import StuffDocumentsChain, MapReduceDocumentsChain, ReduceDocumentsChain, LLMChain

llm = CortexLLM(model=model, max_retries=2, retry_delay=0)


map_chain = LLMChain(llm=llm, prompt=map_template)
reduce_chain = LLMChain(llm=llm, prompt=reduce_template)

combine_docs_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, 
    document_variable_name="summaries"
)

reduce_documents_chain = ReduceDocumentsChain(
    combine_documents_chain=combine_docs_chain,
    collapse_documents_chain=combine_docs_chain,
    token_max=28000,
)

map_reduce_chain = MapReduceDocumentsChain(
    llm_chain=map_chain,
    reduce_documents_chain=reduce_documents_chain,
    document_variable_name="cases",
    return_intermediate_steps=True,
)

In [ ]:
len(texts)

In [ ]:
from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache
import tiktoken
import os
os.environ["TRANSFORMERS_CACHE"] = "/tmp/"
tiktoken_cache_dir = '/tmp/'
os.environ["TIKTOKEN_CACHE_DIR"] = tiktoken_cache_dir
if not os.path.exists(tiktoken_cache_dir):
    raise FileNotFoundError(f"Tokenizer files not found in {tiktoken_cache_dir}")

# Load the specific encoding by name, such as 'cl100k_base'
encoding_name = 'cl100k_base'
tokenizer = tiktoken.get_encoding(encoding_name)
set_llm_cache(InMemoryCache())

# Use the map_reduce_chain as before
result = map_reduce_chain.invoke(texts)

# Print results
print(result["intermediate_steps"])
print(result["output_text"])



In [ ]:
from snowflake.snowpark.types import StructType, StructField, StringType, DateType, TimestampType, VariantType
import datetime

schema = StructType([
    StructField("datetime", TimestampType()),
    StructField("day", DateType()),
    StructField("output_text", StringType()),
    StructField("intermediate_steps", VariantType())
])

# Current datetime and date
current_datetime = datetime.datetime.now()
current_date = current_datetime.date()

data = [(current_datetime, current_date, result["output_text"], result["intermediate_steps"])]
df = session.create_dataframe(data, schema=schema)

df.write.save_as_table("SUPPORT_TICKET_SUMMARY", mode="overwrite")
df.show()

In [ ]:
support_pd = support_tickets.with_column('INDEX_TEXT',
    F.concat(
        F.lit("\n\nCATEGORY: "),
        F.col("CATEGORY"),
        F.lit("##### \nCASE SUBJECT: "), 
        F.col("CASE_TITLE"), 
        F.lit("\n\nCASE DESCRIPTION: "), 
        F.col("CASE_DESCRIPTION"), 
        F.lit('\n\nCASE STATUS: '), 
        F.col("STATUS")
    ))
support_pd.printSchema()


In [ ]:
support_pd.write.save_as_table("SUPPORT_CASES", mode="overwrite")

In [ ]:
support_pd.show()

In [ ]:
session.sql(f"""CREATE OR REPLACE CORTEX SEARCH SERVICE SUPPORT_SERVICE
ON INDEX_TEXT 
WAREHOUSE = "SUPPORT_WH"
TARGET_LAG = '1 day'
AS (
    SELECT INDEX_TEXT, DATE_CREATED, CASE_TITLE, CASE_ID FROM SUPPORT_CASES
)""")

In [ ]:
session.sql("show cortex search services in schema").show()